In [1]:
import yfinance as yf
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

from AA import DataDownloader, AssetAllocation

In [2]:
downloader = DataDownloader()

benchmark = '^GSPC'
assets = ['AAPL', 'IBM', 'TSLA', 'GOOG', 'NVDA']  
start_date = '2019-01-01'
end_date = '2023-12-31'
rf = .065

asset_prices, benchmark_prices = downloader.download_data(start_date= start_date, end_date= end_date,
                                                          assets= assets, benchmark=benchmark)

[*********************100%%**********************]  5 of 5 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
Asset_allocation = AssetAllocation(asset_prices = asset_prices,benchmark_prices = benchmark_prices, rf=rf)

In [4]:
optimizations = Asset_allocation.Optimize_Portfolio(method = "SLSQP")
optimizations

,AAPL,GOOG,IBM,NVDA,TSLA,Optimized Value
Max Sharpe,0.212822,0.010000,0.010000,0.507455,0.259723,1.269909
Max (Smart) Sharpe,0.010000,0.010000,0.010000,0.569682,0.400318,1.200539
Max Omega,0.367592,0.018850,0.313467,0.213552,0.086539,1.348117
Max (Smart) Omega,0.010000,0.010000,0.201554,0.125307,0.653139,1.205929
Min VaR (Empirical),0.010000,0.010000,0.010000,0.389239,0.580761,0.049680
Min VaR (Parametric),0.156482,0.217159,0.606359,0.010000,0.010000,0.024424
Semivariance,0.200000,0.200000,0.200000,0.200000,0.200000,0.000401
Safety-First,0.221457,0.010000,0.010000,0.501680,0.256863,0.079996
Max Sortino,0.010000,0.010000,0.010000,0.747532,0.222468,0.115912
Risk Parity,0.200000,0.200000,0.200000,0.200000,0.200000,0.000014


Usando el mismo timeframe y activos calculamos las metricas con los pesos datos por la optimizacion y verificamos el valor del porceso manual y la class AssetAllocation

### Sharpe 

In [5]:
Weights_Sharpe, Value_Sharpe = optimizations.loc["Max Sharpe"][:-1], optimizations.loc["Max Sharpe"][-1]
weights = Weights_Sharpe

#retornos de activos
returns = asset_prices.pct_change().dropna()
returns.head()

,AAPL,GOOG,IBM,NVDA,TSLA
Date,,,,,
2019-01-03,-0.099607,-0.028484,-0.019964,-0.060417,-0.031472
2019-01-04,0.042689,0.053786,0.039058,0.064068,0.057697
2019-01-07,-0.002226,-0.002167,0.007075,0.052940,0.054361
2019-01-08,0.019063,0.007385,0.014219,-0.024895,0.001164
2019-01-09,0.016982,-0.001505,0.007177,0.019667,0.009483


In [6]:
#retornos del portafolio
portfolio_returns = returns.dot(weights)
portfolio_returns.head()

Date
2019-01-03   -0.060516
2019-01-04    0.057511
2019-01-07    0.040559
2019-01-08   -0.008058
2019-01-09    0.016114
dtype: float64

In [7]:
#retornos menos la tasa libre de riesgo diaria
excess_returns_daily = portfolio_returns - (rf / 252)
excess_returns_daily


Date
2019-01-03   -0.060774
2019-01-04    0.057253
2019-01-07    0.040301
2019-01-08   -0.008316
2019-01-09    0.015856
                ...   
2023-12-22   -0.004947
2023-12-26    0.008062
2023-12-27    0.006081
2023-12-28   -0.006905
2023-12-29   -0.006271
Length: 1257, dtype: float64

In [8]:
# Anualizamos el promedio  de la resta para obtener rendimiento promedio anual del portafolio
excess_returns_annualized = excess_returns_daily.mean() * 252
excess_returns_annualized

0.5444137309581191

In [9]:
# Calculamos volatilidad anual del portafolio
portfolio_volatility = portfolio_returns.std() * np.sqrt(252)
portfolio_volatility 

0.42870305271303283

In [10]:
# Calculamos Sharpe
sharpe_ratio = excess_returns_annualized / portfolio_volatility
sharpe_ratio

1.2699086874068546

In [11]:
# Comparamos
round(sharpe_ratio - Value_Sharpe, 10)

-0.0

**Diferencia menor a $ 1e^{-10}$**

### Omega

In [12]:
Weights_Omega, Value_Omega = optimizations.loc["Max Omega"][:-1], optimizations.loc["Max Omega"][-1]
weights = Weights_Omega

weights

AAPL    0.367592
GOOG    0.018850
IBM     0.313467
NVDA    0.213552
TSLA    0.086539
Name: Max Omega, dtype: float64

In [13]:
# Retornos de activos
returns = asset_prices.pct_change().dropna()
portfolio_returns = pd.DataFrame(returns.dot(weights)) 
portfolio_returns

,0
Date,
2019-01-03,-0.059035
2019-01-04,0.047624
2019-01-07,0.017368
2019-01-08,0.006388
2019-01-09,0.013484
...,...
2023-12-22,-0.000629
2023-12-26,0.004396
2023-12-27,0.002715


In [14]:
# Calcular los retornos diarios del benchmark
benchmark_returns = benchmark_prices.pct_change().dropna()  
benchmark_returns 

,^GSPC
Date,
2019-01-03,-0.024757
2019-01-04,0.034336
2019-01-07,0.007010
2019-01-08,0.009695
2019-01-09,0.004098
...,...
2023-12-22,0.001660
2023-12-26,0.004232
2023-12-27,0.001430


In [15]:
# Obtener diferencia del portafolio respecto al benchmark
excess_returns = portfolio_returns[0] -  benchmark_returns[benchmark_returns.columns[0]] 
excess_returns

Date
2019-01-03   -0.034279
2019-01-04    0.013289
2019-01-07    0.010358
2019-01-08   -0.003307
2019-01-09    0.009386
                ...   
2023-12-22   -0.002289
2023-12-26    0.000164
2023-12-27    0.001285
2023-12-28   -0.001297
2023-12-29   -0.001203
Length: 1257, dtype: float64

In [16]:
positive_excess = excess_returns[excess_returns > 0].sum()
negative_excess = -excess_returns[excess_returns < 0].sum()
positive_excess, negative_excess

(4.787348010463608, 3.5511364726799446)

In [17]:
omega_ratio = positive_excess / negative_excess
omega_ratio

1.3481171583503602

In [18]:
# Comparamos
round(omega_ratio - Value_Omega, 10)

0.0

**Diferencia menor a $ 1e^{-10}$**

### Safety First Ratio

In [19]:
Weights_SFRatio, Value_SFRatio = optimizations.loc["Safety-First"][:-1], optimizations.loc["Safety-First"][-1]
weights = Weights_SFRatio

weights

AAPL    0.221457
GOOG    0.010000
IBM     0.010000
NVDA    0.501680
TSLA    0.256863
Name: Safety-First, dtype: float64

In [20]:
# Retorno del portafolio
returns = asset_prices.pct_change().dropna()
portfolio_returns = pd.DataFrame(returns.dot(weights)) 
portfolio_returns

,0
Date,
2019-01-03,-0.060937
2019-01-04,0.057344
2019-01-07,0.040078
2019-01-08,-0.007753
2019-01-09,0.016120
...,...
2023-12-22,-0.004696
2023-12-26,0.008197
2023-12-27,0.006273


In [21]:
# Retorno del portafolio
portfolio_return = portfolio_returns.mean().item()
portfolio_return

0.002407829565697128

In [22]:
# Retorno Mínimo Aceptable (MAR)
MAF = rf / 252
MAF

0.00025793650793650796

In [23]:
# Volatilidad del portafolio
cov_matrix = returns.cov()
portfolio_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
portfolio_vol

0.026875169300777876

In [24]:
# Calculamos el SFratio
SFratio = (portfolio_return - MAF) / portfolio_vol

print(f"Safety First Ratio: {SFratio}")

Safety First Ratio: 0.0799955168170194


In [25]:
# Comparamos
round(SFratio - Value_SFRatio, 10)

-0.0

**Diferencia menor a $ 1e^{-10}$**